# Point Correspondence
The correspondence generation process is a crucial step in the Fast-Global-Registration algorithm, where the goal is to establish reliable matches between two sets of points from partially overlapping surfaces. This process enables the alignment of these surfaces by identifying pairs of points that likely correspond to the same location in both point sets. Given the potential presence of noise, partial overlaps, and outliers in real-world data, the correspondence generation process involves multiple stages to filter and refine the matches. These stages include collecting initial matches, verifying mutual agreement through a reciprocity test, and ensuring geometric consistency using a tuple test. By progressively refining the set of correspondences, we can improve the accuracy and robustness of the overall registration process.
This notebook includes a visualization and explanation of the correspondences generation step in the Fast-Global-Registration algorithm.

<b>NOTE</b>: The visualizations open a new window in your computer, in order to continue running, the visualization window have to be closed. 

## Setup
Run the following cells to set up the notebook

In [4]:
import os
import sys

import open3d as o3d
from ipywidgets import widgets

sys.path.append("..")
from demo.utils.visualizers import visualize_correspondences
from src.logic.correspondence import collect_all_correspondences, reciprocity_test, tuple_test
sys.path.pop()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


'..'

### Choose an Example
Before running the next cells, you can choose any of the examples provided in the dataset folder using the following dropdown selector.

In [5]:
dropdown = widgets.Dropdown(
    options=os.listdir('../dataset'),
    description='Example:',
    value="pairwise_no_noise_01_rot_05",
    layout={'width': 'max-content'},
    disabled=False,
)
dropdown

Dropdown(description='Example:', index=50, layout=Layout(width='max-content'), options=('pairwise_noise_xyz_le…

## Original Point Clouds
Run the following cells to view the original point clouds (the first point cloud is displayed in red while the second is displayed in blue)

In [6]:
pcd1 = o3d.io.read_point_cloud(f"../dataset/{dropdown.value}/Depth_0000.ply")
pcd2 = o3d.io.read_point_cloud(f"../dataset/{dropdown.value}/Depth_0001.ply")

In [7]:
pcd1.paint_uniform_color([1, 0, 0])
pcd2.paint_uniform_color([0, 0, 1])
o3d.visualization.draw_geometries([pcd1, pcd2], window_name="Original Point Clouds")

## 1. Collecting All Matches
The first step in the point correspondence search is to collect candidate matches between two point sets $\( P \)$ and $\( Q \)$. For each point set, we first extract features that describe the local geometric properties around each point. In Fast-Global-Registration the Fast Point Feature Histogram (FPFH), are used to represent points in a high-dimensional feature space. 

Once features are extracted, we perform nearest neighbor queries in this feature space rather than directly in the original 3D space. This approach helps to find points that have similar local geometries, making the matching process more robust to noise and partial overlaps. For each point $\( p \)$ in $\( P \)$, we find its nearest neighbor in $\( Q \)$'s feature space, and vice versa. This process establishes a set of initial correspondences $\( K_I \)$ between the two point sets. These initial matches may contain a high number of outliers, so further filtering is required.


In [8]:
kappa1 = collect_all_correspondences(pcd1, pcd2)
print(f"Found {len(kappa1)} matches")

Found 32560 matches


In [9]:
visualize_correspondences(pcd1, pcd2, kappa1, "All Correspondences")

## 2. Reciprocity Test
After collecting all matches, we perform a **reciprocity test** to filter out unreliable correspondences. A correspondence pair $\( (p, q) \)$ is kept only if $\( p \)$ is the nearest neighbor of $\( q \)$ in $\( P \)$ and $\( q \)$ is the nearest neighbor of $\( p \)$ in $\( Q \)$. This step ensures that both points mutually agree on being the best match, resulting in a refined set of correspondences $\( K_{II} \)$.

In [10]:
kappa2 = reciprocity_test(kappa1)
print(f"There are {len(kappa2)} after the reciprocity test")

There are 2805 after the reciprocity test


In [11]:
visualize_correspondences(pcd1, pcd2, kappa2, "After Reciprocity Test")

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


## 3. Tuple Test
The **tuple test** is the next step to further filter the correspondences. This test checks for geometric consistency among sets of correspondence pairs. We randomly select 3 pairs of correspondences $\( (p_1, q_1), (p_2, q_2), (p_3, q_3) \)$ from $\( K_{II} \)$ and verify that the ratios of distances between corresponding points in each pair are consistent. Specifically, the condition $\( \tau < \frac{\|p_i - p_j\|}{\|q_i - q_j\|} < \frac{1}{\tau} \)$ for $\( \tau = 0.9 \)$ must be satisfied for each combination $\( i \neq j \)$. This test retains only the geometrically compatible correspondences, forming the final set $\( K_{III} \)$.

In [12]:
kappa3 = tuple_test(kappa2, pcd1, pcd2)
print(f"There are {len(kappa3)} after the tuple test")

There are 1260 after the tuple test


In [13]:
visualize_correspondences(pcd1, pcd2, kappa3, "After Tuple Test")